# Introduction

**Authors: M. Ravasi, D. Vargas, I. Vasconcelos**

Welcome to the **Solving large-scale inverse problems in Python with PyLops** tutorial!

The aim of this tutorial is to:

- introduce you to the concept of *linear operators* and their usage in the solution of *inverse problems*;
- show how PyLops can be used to set-up non-trivial linear operators and solve inverse problems in Python; 
- Walk you through a set of use cases where PyLops has been leveraged to solve real scientific problems and present future directions of development.

## Useful links

- Tutorial Github repository: https://github.com/mrava87/pylops_pydata2020
        
- PyLops Github repository: https://github.com/equinor/pylops

- PyLops reference documentation: https://pylops.readthedocs.io/en/latest/

## Theory in a nutshell

In this tutorial we will try to keep the theory to a minimum and quickly expose you to practical examples. However, we want to make sure that some of the basic underlying concepts are clear to everyone and define a common mathematical notation.

At the core of PyLops lies the concept of **linear operators**. A linear operator is generally a mapping or function that acts linearly on elements of a space to produce elements of another space. More specifically we say that $\mathbf{A}:\mathbb{F}^m \to \mathbb{F}^n$ is a linear operator that maps a vector of size $m$ in the *model space* to a vector of size $n$ in the *data space*:

$$\mathbf{y} = \mathbf{A} \mathbf{x}$$

We will refer to this as **forward model (or operation)**. 

Conversely the application of its adjoint to a data vector is referred to as **adjoint modelling (or operation)**:

$$\mathbf{x} = \mathbf{A}^H \mathbf{y}$$

In its simplest form, a linear operator can be seen as a **matrix** of size $n \times m$ (and the adjoint is simply its transpose and complex conjugate). However in a more general sense we can think of a linear operator as any pair of software code that mimics the effect a matrix on a model vector as well as that of its adjoint to a data vector.

Solving an inverse problems accounts to removing the effect of the operator/matrix $\mathbf{A}$ from the data $\mathbf{y}$ to retrieve the model $\mathbf{x}$ (or an approximation of it).

$$\hat{\mathbf{x}} = \mathbf{A}^{-1} \mathbf{y}$$

In practice, the inverse of $\mathbf{A}$ is generally not explicitely required. A solution can be obtained using either direct methods, matrix decompositions (eg SVD) or iterative solvers. Luckily, many iterative methods (e.g. cg, lsqr) do not need to know the individual entries of a matrix to solve a linear system. Such solvers only require the computation of forward and adjoint matrix-vector products - exactly what a linear operator does!

**So what?**
We have learned that to solve an inverse problem, we do not need to express the modelling operator in terms of its dense (or sparse) matrix. All we need to know is how to perform the forward and adjoint operations - ideally as fast as possible and using the least amount of memory. 

Our first task will be to understand how we can effectively write a linear operator on pen and paper and translate it into computer code. We will consider 2 examples:

- Element-wise multiplication (also known as Hadamard product)
- First Derivative

Let's first import the libraries we need in this tutorial

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pylops
import scooby

from scipy.linalg import lstsq
from pylops import LinearOperator
from pylops.utils import dottest

## Element-wise multiplication

We start by creating a barebore linear operator that performs a simple element-wise multiplication between two vectors (the so-called Hadamart product):

$$ y_i = d_i x_i  \quad \forall i=0,1,...,n-1 $$

If we think about the forward problem the way we wrote it before, we can see that this operator can be equivalently expressed as a dot-product between a square matrix $\mathbf{D}$ that has the $d_i$ elements along its main diagonal and a vector $\mathbf{x}$:

$$\mathbf{x} = \mathbf{D} \mathbf{y}$$

Because of this, the related linear operator is called *Diagonal* operator in PyLops.

We are ready to implement this operator in 2 different ways:

- directly as a diagonal matrix; 
- as a linear operator that performs directly element-wise multiplication.

### Dense matrix definition

In [ ]:
n = 10
diag = np.arange(n)

D = np.diag(diag)
print('D:\n', D)

We can now apply the forward by simply using `np.dot`

In [ ]:
x = np.ones(n)
y = np.dot(D, x) # or D.dot(x) or D @ x
print('y: ', y)

As we have access to all the entries of the matrix, it is very easy to write the adjoint

In [ ]:
xadj = np.dot(np.conj(D.T), y)
print('xadj: ', xadj)

*Note:* since the elements of our matrix are real numbers, we can avoid applying the complex conjugation here.

Everything seems very easy so far. This approach does however carry some problems:
    
- we are storing $N^2$ numbers, even though we know that our matrix has only elements along its diagonal.
- we are applying a dot product which requires $N^2$ multiplications and summations (most of them with zeros)

Of course in this case we could use a sparse matrix, which allows to store only non-zero elements (and their index) and provides a faster way to perform the dot product.

### Linear operator definition

Let's take a leap of faith, and see if we can avoid thinking about the matrix altogether and write just an equivalent (ideally faster) piece of code that mimics this operation.

To write its equivalent linear operator, we define a class with an init method, and 2 other methods:
    
- _matvec: we write the forward operation here
- _rmatvec: we write the adjoint operation here
    
We see that we are also subclassing a PyLops LinearOperator. For the moment let's not get into the details of what that entails and simply focus on writing the content of these three methods.

In [ ]:
class Diagonal(LinearOperator):
    """Short version of a Diagonal operator. See
    https://github.com/equinor/pylops/blob/master/pylops/basicoperators/Diagonal.py
    for a more detailed implementation
    """
    def __init__(self, diag, dtype='float64'):
        self.diag = diag
        self.shape = (len(self.diag), len(self.diag))
        self.dtype = np.dtype(dtype)

    def _matvec(self, x):
        y = self.diag * x
        return y

    def _rmatvec(self, x):
        y = np.conj(self.diag) * x
        return y

Now we create the operator

In [ ]:
Dop = Diagonal(diag)
print('Dop: ', Dop)

### Linear operator application

Forward

In [ ]:
y = Dop * x # Dop @ x
print('y: ', y)

Adjoint

In [ ]:
xadj = Dop.H * y
print('xadj: ', xadj)

As expected we obtain the same results!

**EX:** try making a much bigger vector $\mathbf{x}$ and time the forward and adjoint for the two approaches

In [ ]:
def Diagonal_timing():
    """Timing of Diagonal operator
    """
    n = 10000
    diag = np.arange(n)
    x = np.ones(n)

    # dense
    D = np.diag(diag)

    from scipy import sparse
    Ds = sparse.diags(diag, 0)

    # lop
    Dop = Diagonal(diag)

    # uncomment these
    %timeit -n3 -r3 np.dot(D, x)
    %timeit -n3 -r3 Ds.dot(x)
    %timeit -n3 -r3 Dop._matvec(x)

In [ ]:
Diagonal_timing()

### Linear operator testing

One of the most important aspect of writing a Linear operator is to be able to verify that the code implemented in forward mode and the code implemented in adjoint mode are effectively adjoint to each other. 

If this is not the case, we will struggle to invert our linear operator - some iterative solvers will diverge and other show very slow convergence.

This is instead the case if the so-called *dot-test* is passed within a certain treshold:

$$
(\mathbf{A}*\mathbf{u})^H*\mathbf{v} = \mathbf{u}^H*(\mathbf{A}^H*\mathbf{v})
$$

where $\mathbf{u}$ and $\mathbf{v}$ are two random vectors.

Let's use `pylops.utils.dottest`

In [ ]:
dottest(Dop, n, n, verb=True);

## First Derivative

Let's consider now something less trivial. We use a first-order centered first derivative stencil:

$$ y_i = \frac{x_{i+1} - x_{i-1}}{2 \Delta}  \quad \forall i=1,2,...,N $$

where $\Delta$ is the sampling step of the input signal. Note that we will deal differently with the edges, using a forward/backward derivative.

### Dense matrix definition

In [ ]:
nx = 11

D = np.diag(0.5*np.ones(nx-1), k=1) - np.diag(0.5*np.ones(nx-1), k=-1) 
D[0, 0] = D[-1, -2] = -1
D[0, 1] = D[-1, -1] = 1
print('D:\n', D)

### Linear operator definition

**EX:** try writing the operator

In [ ]:
# class FirstDerivative(LinearOperator):
    def __init__(self, ...):
        # fill here

    def _matvec(self, x):
        # fill here

    def _rmatvec(self, x):
        # fill here

**SOLUTION**

In [ ]:
class FirstDerivative(LinearOperator):
    """Short version of a FirstDerivative operator. See
    https://github.com/equinor/pylops/blob/master/pylops/basicoperators/FirstDerivative.py
    for a more detailed implementation
    """
    def __init__(self, N, sampling=1., dtype='float64'):
        self.N = N
        self.sampling = sampling
        self.shape = (N, N)
        self.dtype = dtype
        self.explicit = False

    def _matvec(self, x):
        x, y = x.squeeze(), np.zeros(self.N, self.dtype)
        y[1:-1] = (0.5 * x[2:] - 0.5 * x[0:-2]) / self.sampling
        # edges
        y[0] = (x[1] - x[0]) / self.sampling
        y[-1] = (x[-1] - x[-2]) / self.sampling
        return y

    def _rmatvec(self, x):
        x, y = x.squeeze(), np.zeros(self.N, self.dtype)
        y[0:-2] -= (0.5 * x[1:-1]) / self.sampling
        y[2:] += (0.5 * x[1:-1]) / self.sampling
        # edges
        y[0] -= x[0] / self.sampling
        y[1] += x[0] / self.sampling
        y[-2] -= x[-1] / self.sampling
        y[-1] += x[-1] / self.sampling
        return y

Define the operator

In [ ]:
Dop = FirstDerivative(nx)
print('Dop: ', Dop)

Perform the dot test

In [ ]:
dottest(Dop, nx, nx, verb=True);

Now that you understand, you can use PyLops implementation of this operator (see https://pylops.readthedocs.io/en/latest/api/generated/pylops.FirstDerivative.html for details)

In [ ]:
Dop = pylops.FirstDerivative(nx, edge=True)
print('Dop: ', Dop)

In [ ]:
dottest(Dop, nx, nx, verb=True);

### Linear operator application

In [ ]:
x = np.arange(nx) - (nx-1)/2
print('x: ', x)

Forward

In [ ]:
y = np.dot(D, x)
print('y: ', y)

y = Dop * x
print('y: ', y)

Adjoint

In [ ]:
xadj = np.dot(D.T, y)
print('xadj: ', xadj)

xadj = Dop.H * y
print('xadj: ', xadj)

**EX:** Same as before, let's time our two implementations

In [ ]:
def FirstDerivative_timing():
    """Timing of FirstDerivative operator
    """
    nx = 2001
    x = np.arange(nx) - (nx-1)/2

    # dense
    D = np.diag(0.5*np.ones(nx-1),k=1) - np.diag(0.5*np.ones(nx-1),-1)
    D[0, 0] = D[-1, -2] = -1
    D[0, 1] = D[-1, -1] = 1

    # lop
    Dop = pylops.FirstDerivative(nx, edge=True)

    # uncomment these
    %timeit -n3 -r3 np.dot(D, x)
    %timeit -n3 -r3 Dop._matvec(x)


In [ ]:
FirstDerivative_timing()

**EX:** try to compare the memory footprint of the matrix $\mathbf{D}$ compared to its equivalent linear operator. Hint: install ``pympler`` and use ``pympler.asizeof``

In [ ]:
def FirstDerivative_memory():
    """Memory footprint of Diagonal operator
    """
    from pympler import asizeof
    from scipy.sparse import diags
    nn = (10 ** np.arange(2, 4, 0.5)).astype(np.int)

    mem_D = []
    mem_Ds = []
    mem_Dop = []
    for n in nn:
        D = np.diag(0.5 * np.ones(n - 1), k=1) - np.diag(0.5 * np.ones(n - 1),
                                                         -1)
        D[0, 0] = D[-1, -2] = -1
        D[0, 1] = D[-1, -1] = 1
        Ds = diags((0.5 * np.ones(n - 1), -0.5 * np.ones(n - 1)),
                   offsets=(1, -1))
        Dop = pylops.FirstDerivative(n, edge=True)
        mem_D.append(asizeof.asizeof(D))
        mem_Ds.append(asizeof.asizeof(Ds))
        mem_Dop.append(asizeof.asizeof(Dop))

    plt.figure(figsize=(12, 3))
    plt.semilogy(nn, mem_D, '.-k', label='D')
    plt.semilogy(nn, mem_Ds, '.-b', label='Ds')
    plt.semilogy(nn, mem_Dop, '.-r', label='Dop')
    plt.legend()
    plt.title('Memory comparison')

In [ ]:
FirstDerivative_memory()

Finally, let's try to move on step further and try to solve the inverse problem. 

For the dense matrix, we will use `scipy.linalg.lstsq`. For operator PyLops this can be very easily done by using the '/' which will call `scipy.sparse.linalg.lsqr` solver (you can also use this solver directly if you want to fine tune some of its input parameters

In [ ]:
xinv = lstsq(D, y)[0]
print('xinv: ', xinv)

xinv = Dop / y
print('xinv: ', xinv)

In both cases we have retrieved the correct solution!

## Chaining operators

Up until now, we have discussed how brand new operators can be created in few systematic steps. This sounds cool, but it may look like we would need to do this every time we need to solve a new problem.

This is where **PyLops** comes in. Alongside providing users with an extensive collection of operators, the library allows such operators to be combined via basic algebraic operations (eg summed, subtracted, multiplied) or chained together (vertical and horizontal stacking, block and block diagonal).

We will see more of this in the following. For now let's imagine to have a modelling operator that is a smooth first-order derivative. To do so we can chain the ``FirstDerivative`` operator ($\mathbf{D}$) that we have just created with a smoothing operator ($\mathbf{S}$)(https://pylops.readthedocs.io/en/latest/api/generated/pylops.Smoothing1D.html#pylops.Smoothing1D) and write the following problem:

$$\mathbf{y} = \mathbf{S} \mathbf{D} \mathbf{x}$$

Let's create it first and attempt to invert it afterwards.

In [ ]:
nx = 51
x = np.ones(nx)
x[:nx//2] = -1

Dop = pylops.FirstDerivative(nx, edge=True, kind='forward')
Sop = pylops.Smoothing1D(5, nx)

# Chain the two operators
Op = Sop * Dop
print(Op)

# Create data
y = Op * x

# Invert
xinv = Op / y
xinv = pylops.optimization.leastsquares.NormalEquationsInversion(Op, [pylops.Identity(nx)], y, epsRs=[1e-3,])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 5))
ax1.plot(y, '.-k')
ax1.set_title(r"Data $y$")
ax2.plot(x, 'k', label='x')
ax2.plot(xinv, '--r', label='xinv')
ax2.legend()
ax2.set_title(r"Model $x$")
plt.tight_layout()

## Recap

In this first tutorial we have learned to:

- translate a linear operator from pen and paper to computer code
- write our own linear operators
- use PyLops linear operators to perform forward, adjoint and inverse
- combine PyLops linear operators.

In [ ]:
scooby.Report(core='pylops')